In [47]:
import tensorflow as tf
import coco_helper as ch
from config import *
import numpy as np
import pickle

In [2]:
def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    # If the value is an eager tensor BytesList won't unpack a string from an EagerTensor.
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() 
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
    """Returns a float_list from a float / double."""
    return 

def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [14]:
def serialize_example(idd,image_raw,size,kpts,joints):
    
    kpts=kpts.tobytes()
    joints=joints.tobytes()
    
    feature = {
        'id': _int64_feature(idd),
        'image_raw': _bytes_feature(image_raw),
        'size': tf.train.Feature(float_list=tf.train.FloatList(value=size)),
        'kpts': _bytes_feature(kpts),
        'joints': _bytes_feature(joints),
    }
    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    return example_proto.SerializeToString()

In [5]:
with open(TRANSFORMED_TRAIN_ANNOTATIONS_PATH, 'rb') as f:
    combined_dict = pickle.load(f)

In [33]:
filename="./dataset/transformed/coco_keypoints-001.tfrecords"
writer= tf.io.TFRecordWriter(filename)
count=0
for idd,l in combined_dict.items():
    size=l[0]
    kpts=l[1]
    joints=l[2]
    
    image_raw=tf.io.read_file(ch.id_to_filename(idd))
    
    example = serialize_example(idd,image_raw,size,kpts,joints)
    writer.write(example)
    
    count+=1
    if not count%100: print(".",end="")
    if not count%1000:
        writer.flush()
        writer.close()
        filename=f"./dataset/transformed/coco_keypoints-{int(count/1000):03}.tfrecords"
        writer= tf.io.TFRecordWriter(filename)
        print("|",end="")
writer.flush()
writer.close()       

..........|..........|..........|..........|..........|..........|..........|..........|..........|..........|..........|..........|..........|..........|..........|..........|..........|..........|..........|..........|..........|

KeyboardInterrupt: 

In [20]:
tf.train.Feature(bytes_list=tf.train.BytesList(value=[]))

TypeError: [1, 2, 3] has type list, but expected one of: int, long

In [25]:
s=tf.io.serialize_tensor(tf.constant([1.0,2.0,3.0]))
s

<tf.Tensor: id=38, shape=(), dtype=string, numpy=b'\x08\x01\x12\x04\x12\x02\x08\x03"\x0c\x00\x00\x80?\x00\x00\x00@\x00\x00@@'>

In [26]:
tf.io.parse_tensor(s,out_type=tf.float32)

<tf.Tensor: id=39, shape=(3,), dtype=float32, numpy=array([1., 2., 3.], dtype=float32)>

In [30]:
tr=tf.ragged.constant([[1,2,3],[4,5]],dtype=tf.float32)
tr

<tf.RaggedTensor [[1.0, 2.0, 3.0], [4.0, 5.0]]>

In [41]:
import numpy as np
a=np.arange(12).reshape(2,6)

In [43]:
a.tobytes()

b'\x00\x00\x00\x00\x01\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x04\x00\x00\x00\x05\x00\x00\x00\x06\x00\x00\x00\x07\x00\x00\x00\x08\x00\x00\x00\t\x00\x00\x00\n\x00\x00\x00\x0b\x00\x00\x00'

In [10]:
type(combined_dict[110][2])

numpy.ndarray

In [32]:
f"{int(2000/1000):03}"

'002'

In [35]:
y.flush

NameError: name 'y' is not defined

In [37]:
1 if False else 0

0

In [38]:
import glob

In [156]:
tfrecord_files=glob.glob(TRANSFORMED_TRAIN_ANNOTATIONS_PATH+"-*.tfrecords")

In [247]:
raw_dataset = tf.data.TFRecordDataset(tfrecord_files)

In [234]:
def read_record(serialized_record):
    features=pickle.loads(serialized_record.numpy())
    
    idd=features['id']
    image_raw=features['image_raw']
    size=features['size']
    kpts=features['kpts']
    joints=features['joints']
    
    idd=tf.constant(idd)
    image_raw=tf.constant(image_raw)
    size=tf.constant(size)
    kpts=tf.constant(kpts)
    joints=tf.constant(joints)
    
    return idd,image_raw,size,kpts,joints
        

In [260]:
@tf.py_function
def read_record(serialized_record):
    pt=tf.io.parse_tensor(serialized_record)
    features=pickle.loads(tf.string,pt)
    return features


TypeError: eager_py_func() missing 2 required positional arguments: 'inp' and 'Tout'

In [255]:
parsed_dataset=raw_dataset.map(read_record)

TypeError: in converted code:

    <ipython-input-252-9e4255276c11>:2 read_record  *
        pt=tf.io.parse_tensor(serialized_record)

    TypeError: parse_tensor() missing 1 required positional argument: 'out_type'


In [259]:
tf.io.parse_tensor(next(iter(raw_dataset)),tf.b)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x80 in position 46: invalid start byte

In [244]:
raw_dataset

<TFRecordDatasetV2 shapes: (), types: tf.string>